Класс можно воспринимать как некое пространство имен, в котором записаны свойства и методы - атрибуты класса. В данном приимере 4 аргумента: 2 свойства и 2 метода:

In [1]:
class Point:
    MAX_COORD = 100
    MIN_COORD = 0
    
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def set_coords(self, x, y):
        self.x = x
        self.y = y

Далее, при создании экземпляров класса, атрибуты остаются в класса и не копируются в отдельные экземпляры. Т.е. атрибуты класса являются общими для всех его экземпляров:

In [2]:
pt1 = Point(1, 2)
pt2 = Point(8, 3)

print(pt1.__dict__, pt2.__dict__)

{'x': 1, 'y': 2} {'x': 8, 'y': 3}


Но при этом из экземпляров класса можно спокойно обращаться к атрибутам класса:

In [3]:
pt1.MAX_COORD

100

In [4]:
pt1.set_coords(3, 3)
print(pt1.__dict__)

{'x': 3, 'y': 3}


При обращении к атрибутам класса внутри методов необходимо дать ссылку либо на класс, либо на экземпляр класса (второй вариант предпочтительнее, так как является более универсальным):

In [5]:
class Point:
    MAX_COORD = 100
    MIN_COORD = 0
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def set_coords(self, x, y):
        if (self.MIN_COORD <= x <= self.MAX_COORD) and (self.MIN_COORD <= y <= self.MAX_COORD):
            self.x = x
            self.y = y
        else: raise ValueError('Неверные координаты')

In [6]:
pt1 = Point()

In [7]:
pt1.set_coords(1, 2)

Но если мы захотим поменять через экземпляр класса какой либо параметр класса таким образом, определив метод set_bound:

In [8]:
class Point:
    MAX_COORD = 100
    MIN_COORD = 0
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def set_coords(self, x, y):
        if (self.MIN_COORD <= x <= self.MAX_COORD) and (self.MIN_COORD <= y <= self.MAX_COORD):
            self.x = x
            self.y = y
        else: raise ValueError('Неверные координаты')
            
    def set_bound(self, left, right):
        self.MAX_COORD = left
        self.MIN_COORD = right

In [9]:
pt = Point()

То при попытке совершить следующее действие:

In [10]:
pt.set_bound(200, -100)

Атрибут класса непоменяется, а в локальной области видимости экземпляра класса будет созданы такие же свойства с новым значением:

In [11]:
Point.__dict__

mappingproxy({'__module__': '__main__',
              'MAX_COORD': 100,
              'MIN_COORD': 0,
              '__init__': <function __main__.Point.__init__(self, x=0, y=0)>,
              'set_coords': <function __main__.Point.set_coords(self, x, y)>,
              'set_bound': <function __main__.Point.set_bound(self, left, right)>,
              '__dict__': <attribute '__dict__' of 'Point' objects>,
              '__weakref__': <attribute '__weakref__' of 'Point' objects>,
              '__doc__': None})

In [12]:
pt.__dict__

{'x': 0, 'y': 0, 'MAX_COORD': 200, 'MIN_COORD': -100}

Так работает оператор присваивания: если он не видит свойства в текущей области видимости, то они в ней создаются. 

Правильно это сделать следущим образом: сделать метод set_bound методом класса, и передавать ему не экземпляр класса, а ссылку на класс:

In [13]:
    class Point:
        MAX_COORD = 100
        MIN_COORD = 0

        def __init__(self, x=0, y=0):
            self.x = x
            self.y = y

        def set_coords(self, x, y):
            if (self.MIN_COORD <= x <= self.MAX_COORD) and (self.MIN_COORD <= y <= self.MAX_COORD):
                self.x = x
                self.y = y
            else: raise ValueError('Неверные координаты')

        @classmethod        
        def set_bound(cls, left, right):
            cls.MAX_COORD = left
            cls.MIN_COORD = right

И затем поменять параметры, вызвав этот метод от класса:

In [14]:
pt = Point()

In [15]:
pt.set_bound(200, -100)

In [16]:
Point.__dict__

mappingproxy({'__module__': '__main__',
              'MAX_COORD': 200,
              'MIN_COORD': -100,
              '__init__': <function __main__.Point.__init__(self, x=0, y=0)>,
              'set_coords': <function __main__.Point.set_coords(self, x, y)>,
              'set_bound': <classmethod at 0x1c07e339160>,
              '__dict__': <attribute '__dict__' of 'Point' objects>,
              '__weakref__': <attribute '__weakref__' of 'Point' objects>,
              '__doc__': None})

In [17]:
pt.__dict__

{'x': 0, 'y': 0}

## Магические методы для атрибутов

setattr(self, key, value) - автоматически вызывается при изменении свойства key класса<br>

getattribute(self, item) - автоматически вызывается при получении свойства класса с именем item<br>

getattr(self, item) - автоматически вызывается при получении несуществующего свойства item класса<br>

delattr(self, item) - автоматически вызывается при удалении свойства item (при этом неважно, существует оно или нет)<br>

### getattribute

In [18]:
class Point:
    MAX_COORD = 100
    MIN_COORD = 0
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def set_coords(self, x, y):
        if (self.MIN_COORD <= x <= self.MAX_COORD) and (self.MIN_COORD <= y <= self.MAX_COORD):
            self.x = x
            self.y = y
        else: raise ValueError('Неверные координаты')
            
    def __getattribute__(self, item):
        print('__getattribute__')
        return object.__getattribute__(self, item)

In [19]:
pt = Point(2, 3)

In [20]:
a = pt.x

__getattribute__


In [21]:
a

2

In [22]:
pt = None

__getattribute__
__getattribute__
__getattribute__
__getattribute__


Как только идет обращение к какому либо атрибуту через экземпляр класса, то срабатывает магический метод getattribute.

Предположим, что мы хотим запретить доступ к какому либо свойству:

In [23]:
class Point:
    MAX_COORD = 100
    MIN_COORD = 0
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def set_coords(self, x, y):
        if (self.MIN_COORD <= x <= self.MAX_COORD) and (self.MIN_COORD <= y <= self.MAX_COORD):
            self.x = x
            self.y = y
        else: raise ValueError('Неверные координаты')
            
    def __getattribute__(self, item):
        if item == 'x':
            raise ValueError('Доступ запрещен')
        else:
            return object.__getattribute__(self, item)

In [24]:
pt = Point(1, 2)

И тогда при попытке обращения к переменной x от экземпляра класса будет генерироваться исключение:

In [25]:
# pt.x

А при обращении к другому свойству вс будет, как прежде:

In [26]:
pt.y

2

### setattr

Метод setattr вызывается всегда, когда происходит присвоение какому либо атрибуту какого либо значения:

In [27]:
class Point:
    MAX_COORD = 100
    MIN_COORD = 0
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def set_coords(self, x, y):
        if (self.MIN_COORD <= x <= self.MAX_COORD) and (self.MIN_COORD <= y <= self.MAX_COORD):
            self.x = x
            self.y = y
        else: raise ValueError('Неверные координаты')
            
    def __getattribute__(self, item):
        return object.__getattribute__(self, item)
        
    def __setattr__(self, key, value):
        print('__setattr__')
        object.__setattr__(self, key, value)

In [28]:
pt = Point(1, 2)

__setattr__
__setattr__


In [29]:
pt.__dict__

{'x': 1, 'y': 2}

In [30]:
pt = None

### getattr

Метод getattr вызывается автоматически каждый раз, когда идет обращение к несуществующему атрибуту экземпляра класса:

In [31]:
class Point:
    MAX_COORD = 100
    MIN_COORD = 0
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def set_coords(self, x, y):
        if (self.MIN_COORD <= x <= self.MAX_COORD) and (self.MIN_COORD <= y <= self.MAX_COORD):
            self.x = x
            self.y = y
        else: raise ValueError('Неверные координаты')
            
    def __getattribute__(self, item):
        return object.__getattribute__(self, item)
        
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)
        
    def __getattr__(self, item):
        print('__getattr__')

In [32]:
pt = Point(1, 4)

In [33]:
pt.x

1

In [34]:
print(pt.e)

__getattr__
None


Если обратиться к атрибуту класса, этот метод задействован не будет:

In [35]:
print(pt.MAX_COORD)

100


Даже не смотря на то, что это элемент класса.

С помощью этого метода можно определить какую либо конструкцию для этого случая. Например, чтобы метод возвращал False:

In [36]:
class Point:
    MAX_COORD = 100
    MIN_COORD = 0
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def set_coords(self, x, y):
        if (self.MIN_COORD <= x <= self.MAX_COORD) and (self.MIN_COORD <= y <= self.MAX_COORD):
            self.x = x
            self.y = y
        else: raise ValueError('Неверные координаты')
            
    def __getattribute__(self, item):
        return object.__getattribute__(self, item)
        
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)
        
    def __getattr__(self, item):
        return False

In [37]:
pt = Point()

In [38]:
pt.x

0

Теперь при обращении к несуществующему атрибуту на выходе будет False:

In [39]:
pt.xx

False

Без этого метода генерировалось бы исключение AttributeError

### delattr

Метод delattr вызывается, когда удаляется определенный атрибут из экземпляра класса:

In [40]:
class Point:
    MAX_COORD = 100
    MIN_COORD = 0
    
    def __init__(self, x=0, y=0):
        self.x = x
        self.y = y
    
    def set_coords(self, x, y):
        if (self.MIN_COORD <= x <= self.MAX_COORD) and (self.MIN_COORD <= y <= self.MAX_COORD):
            self.x = x
            self.y = y
        else: raise ValueError('Неверные координаты')
            
    def __getattribute__(self, item):
        return object.__getattribute__(self, item)
        
    def __setattr__(self, key, value):
        object.__setattr__(self, key, value)
        
    def __getattr__(self, item):
        return False
    
    def __delattr__(self, item):
        print (f'__delattr__: {item}')
        object.__delattr__(self, item)

In [41]:
pt = Point(1, 2)

In [42]:
del pt.x

__delattr__: x


In [43]:
pt.__dict__

{'y': 2}